# Tacotron2: Natural TTS Synthesis by Conditioning Wavenet on Mel Spectrogram Predictions

- **Tacotron2**: **Tacotron** 구조와 **WaveNet** 구조를 활용하여 더 좋은 음성 합성 성능을 가지는 **TTS** 모델

# Model Architecture

- **모델 전체 구조**
<img src="01.png" width=70% height=70%>

    - Tacotron 모델과는 달리 CBHG Module과 GRU 대신 **Vanilla LSTM**과 **Convolutional Layers**를 사용해서 훨씬 단순한 모델이 되었다.

## 1. Encoder

- **Encoder 전체 구조**
<img src="02.png" width=40% height=40%>

## 2. Attention

- **Location-Sensitive Attention**: Tacotron에서 사용한 **Bahdanau (Additive) Attention**에서 연장하여 이전 time step의 Attention Weights를 현재 time step에서 활용하는 Attention 기법
    - **Attention 전체 구조**
    <img src="03.png" width=70% height=70%>
    
    - 식은 다음과 같다.
        1. Score: $\text{Score}(\mathbf{d}_{t}, \mathbf{h}) = \mathbf{W}^{T} \tanh (\mathbf{W_{d}}\mathbf{d}_{t} + \mathbf{W_{h}}\mathbf{h} + \mathbf{W_{f}}\mathbf{f}_{t} + \mathbf{b})$  
        ($t-1$ step에서의 Attention Weight $\mathbf{a}_{t-1}$와 가중치 행렬(filter) $\mathbf{F}$에 대하여 $\mathbf{f}_{t} = \mathbf{F} * \mathbf{a}_{t-1}$)
            - Bahdanau Additive Attention의 Score는 다음과 같다. 위와 비교해보자.
                - Score: $\text{Score}(\mathbf{d}_{t}, \mathbf{h}) = \mathbf{W}^{T} \tanh (\mathbf{W_{d}}\mathbf{d}_{t}+\mathbf{W_{h}}\mathbf{h})$
        2. Attention Weight: $a_{t, j} = \frac{\exp(\text{Score}(\mathbf{d}_{t}, {h}_{j}))}{\sum_{i=1}^{n} \exp(\text{Score}(\mathbf{d}_{t}, h_{i}))}$
        3. Context: $\mathbf{c}_{t} = \sum_{j=1}^{n} a_{t, j} h_{j}$  
        ($\mathbf{d}_{t}$: $t$ step에서의 Query 벡터, $\mathbf{h}$: Encoder의 output)
    - Location-Sensitive Attention은 기존의 Additive Attention에서 <u>불필요한 반복되거나 필요한 부분이 무시되는 시퀀스가 발생하는 현상을 완화</u>시켜준다.

## 3. Decoder

- **Decoder 전체 구조**
<img src="04.png" width=70% height=70%>

    - **Zoneout**: LSTM에서 사용되는 드롭아웃 기반의 정규화 기술
    <img src="05.png" width=50% height=50%>
    
        - $d$를 dropout mask라 할 때, 식은 다음과 같다.
        <img src="06.png" width=70% height=70%>
        
        - NVIDIA는 Zoneout을 코드로 구현하지 않았다. (추가가 가능하다고 언급함.)
        
    - **Stop Token**: 추론 단계에서 Mel-Spectrogram의 생성을 멈출지 결정하는 역할을 하는 Token
        - Stop Token은 sigmoid 활성화 함수를 통과하여 0에서 1 사이의 값을 갖고, **threshold**(보통 0.5)를 넘으면 생성을 멈추도록 한다.
    - **Teacher Forcing**: 학습 시 Ground-truth Mel-Spectrogram이 Decoder의 input으로 들어간다.

## 4. WaveNet Vocoder

- 먼저 PixelCNN++ 논문에서 제시된 **Mixture of logistic distribution(MoL)**를 알아보자.

## 4.1. Mixture of Logistic Distribution(MoL)

- **Discretized Logistic Mixture Likelihood**: Mixture of logistic distribution을 이용한 이산 데이터를 예측하는 기법
    - **Mixture of logistic distribution**: 로지스틱 분포의 혼합, **로지스틱 분포**는 로지스틱 함수를 누적분포함수로 갖는 분포이다.
        - 로지스틱 함수의 식과 그림은 다음과 같다. (시그모이드 함수는 로지스틱 함수의 일종이다.)
        $$ F(x; \mu, s) = \frac{1}{1 + e^{-(x-\mu)/s}}$$
        <img src="07.png" width=30% height=30%>
        
        - 로지스틱 분포의 식과 그림은 다음과 같다.
        $$ f(x; \mu, s) = \frac{e^{-(x-\mu)/s}}{s(1 + e^{-(x-\mu)/s})^{2}}$$
        <img src="08.png" width=30% height=30%>
        
    - 그림을 생성할 때 Pixel의 색상마다 256개(0~256)의 정수를 예측하게 된다.
    - 이때 Softmax 함수를 이용하여 해당 픽셀을 128이라 예측한다면, 이 모델은 이 값이 127과 129에 가깝다는 것을 모른다.  
    (그 관계를 학습하지 않았기 때문이다.)
        - 예를 들어, Softmax를 통과한 확률 분포가 [0.05, ..., **0**, **0.9**, **0**, ..., 0.05]와 같이 나타날 수 있다.
        
    - 이 문제를 해결하기 위해 Pixel 색상의 256개의 정수가 어떤 연속함수의 확률분포를 형성한다고 가정한다.
        - 이때 이 연속함수를 **로지스틱의 분포의 혼합**으로 둔다.
        <img src="09.png" width=50% height=50%>
        ($\nu$: Pixel 색상의 확률 분포, $\mu$: 모수, 평균(mean), $s$: 모수, 로그 스케일(scale), $\pi$: 모수, 혼합 가중치(mixture weight), $x$: Pixel의 정수 값, $K$: 로지스틱 분포 혼합 개수)
        
    - 이 모수들을 학습하면, 1) 연속적인 분포를 학습할 수 있고, 2) $K=5$로도 충분하여 학습할 파라미터 수($K=5$인 경우 15개)도 적어진다.
    - **loss**는 **Negative Log-Likelihood**를 이용한다.
    
    - 실제 CIFAR-10 데이터의 픽셀의 색상(sub-pixel)값 분포는 다음과 같다. (이 분포를 학습하고자 하는 것이다.)
    <img src="10.png" width=50% height=50%>

## 4.2. WaveNet Vocoder Architecture

- **WaveNet Vocoder 전체 구조**
<img src="11.png" width=70% height=70%>

    - **Input**: Ground truth의 waveform (추론할 때에는 output의 waveform)
    - **Local Condition**: Tacotron2로부터 생성한 mel-spectrogram
    - **Global Condition**: 화자의 정보 (예: 문재인 대통령은 0, 손석희 아나운서는 1, ...)
    - **MoL Functions**: <u>10개의 MoL</u>을 이용하여 학습하도록 한다. 이로부터 양자화($-2^{15} \sim 2^{15}-1$)된 waveform을 예측한다.  

**[참고 논문]**
1. Tacotron2: https://arxiv.org/abs/1712.05884v2
2. Zoneout: https://arxiv.org/abs/1606.01305v4
3. PixelCNN++: https://arxiv.org/abs/1701.05517

**[참고 자료 / 사진 출처]**
1. Tacotron2 리뷰1: https://joungheekim.github.io/2020/10/08/paper-review/
2. Tacotron2 리뷰2: https://chldkato.tistory.com/176
3. Tacotron2 Github (NVIDIA): https://github.com/NVIDIA/tacotron2/tree/fc0cf6a89a47166350b65daa1beaa06979e4cddf
4. Location-Sensitive Attention: https://paperswithcode.com/method/location-sensitive-attention
5. Zoneout: https://paperswithcode.com/method/zoneout
6. NVIDIA가 구현한 코드에는 Zoneout이 없음: https://github.com/NVIDIA/tacotron2/issues/13
7. NVIDIA Issue: https://github.com/NVIDIA/tacotron2/issues/25
8. Tacotron2 Github (hccho2): https://github.com/hccho2/Tacotron2-Wavenet-Korean-TTS
9. Tacotron2 Github (Rayhane-mamah): https://github.com/Rayhane-mamah/Tacotron-2
10. Wikipedia, Logistic function: https://en.wikipedia.org/wiki/Logistic_function
11. Wikipedia, Logistic distribution: https://en.wikipedia.org/wiki/Logistic_distribution